#GSA Data Analysis Project

In [ ]:
#Load necessary packages to complete this data analysis project
import pandas as pd
import numpy as np

In [ ]:
#Import Source .xlsx Worksheets - Contracts(Obligations) & Transactions(Expenditures)
file_wkshts = pd.read_excel('Round 2 Written Assessment - Data.xlsx', sheet_name=None)

#identify .xlsx sheets
contracts_df = pd.DataFrame(file_wkshts['Contracts'],columns=None)
transactions_df = pd.DataFrame(file_wkshts['Transactions'],columns=None)

#View my df headers - Contracts & Transactions
# print(contracts_df.keys())
# print("\n ")
# print(transactions_df.keys())

In [ ]:
#Display Contracts_df
contracts_df

In [ ]:
#Contracts_df - Remove Trailing spaces on str fields 
contracts_df['ContractID'].str.strip()
contracts_df['Customer'].str.strip()
contracts_df['Category'].str.strip()

#changed DollarsObligated to Float
contracts_df['DollarsObligated'] = contracts_df['DollarsObligated'].astype('float64')

#view Contracts_df after changes
contracts_df

In [ ]:
#Display Transactions_df
transactions_df

In [ ]:
#Remove trailing spaces from str fields
transactions_df['TransactionID'].str.strip()
transactions_df['ContractID'].str.strip()
transactions_df['Description'].str.strip()

#changed DollarsObligated to Float
transactions_df['Price'] = transactions_df['Price'].astype('float64')

#View transactions_df after changes
#transactions_df

In [ ]:
#Merge both dataframes to analyze data
merge_df = pd.merge(contracts_df,transactions_df,how='left',on=['ContractID'])

#Rename Transaction and Contract Date Columns
merge_df2 = merge_df.rename(columns={"Date_x":"Contract_Dt", "Date_y":"Transaction_Dt"},inplace=True)

#View merged df
merge_df

In [ ]:
#print merge_df to csv
#merge_df.to_csv('master_data.csv', index=False)

In [ ]:
#Question 1: Calculate number of transactions for each ContractID and sort transaction count in DESC order
transaction_ct = merge_df.groupby(by=['ContractID'])['TransactionID'].count().sort_values(ascending=False)
transaction_ct

In [ ]:
#Question 2: Calculate Average transaction dollar value on each day in March
avg_transaction = merge_df.groupby(by=['Transaction_Dt'])['Price'].mean().filter(like='2020-03')
avg_transaction

In [ ]:
#Question 3: Calculate what percentage of each customer's associated obligations have been spent as transactions

#Create function to calculate Expended Percentage
def customer_expended(merge_df):
    return merge_df['Price'].sum() / float(merge_df['DollarsObligated'].mean())

#Group by Customer, Category, and ContractID to get the DollarsObligated and Price
cust_activity1 = merge_df.groupby(by=['Customer','Category','ContractID']).agg({'DollarsObligated':np.mean, 'Price':np.sum})


#Apply Expended Percentage by same grouping mentioned above (lines 7-8)
cust_activity2 = merge_df.groupby(by=['Customer','Category','ContractID']).agg({'DollarsObligated':np.mean, 'Price':np.sum}).apply(customer_expended, axis=1)

#Combine both Cust_Activity1 and Cust_Activity2 to have Expended Percentage function together
consolidated_cust_activity = pd.concat([cust_activity1,cust_activity2],axis=1)

#Rename Expended Percentage Column
consolidated_cust_activity.rename(columns={consolidated_cust_activity.columns[2]:'ContractExpendedPercentage'})